In [1]:
import pandas as pd
import numpy as np
import nltk

nltk.download("wordnet")
import re
from bs4 import BeautifulSoup


[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

In [3]:
DATA_PATH = "./data"
DATA_FILE = "amazon_reviews_us_Jewelry_v1_00.tsv"

DATA_COL = "review_body"
TARGET_COL = "star_rating"

RANDOM_SEED = 42

## Read Data


## Keep Reviews and Ratings


In [4]:
# Load the tab separated data file, and print the first 5 rows for confirmation
data = pd.read_csv(f"{DATA_PATH}/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=[TARGET_COL, DATA_COL], low_memory=True)
data.head()


/var/folders/d7/gcxbjppx70qfkdwy32kg04dh0000gn/T/ipykernel_30773/1935721861.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f"{DATA_PATH}/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=[TARGET_COL, DATA_COL])


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


Understanding Data


In [5]:
# Drop NA values
data.dropna(inplace=True)

In [6]:
data["review_body"].isna().values.sum()

0

In [7]:
data.describe()


,star_rating,review_body
count,1766807,1766807
unique,10,1618522
top,5,Love it
freq,1040896,4288


In [8]:
data.star_rating.unique()


array([5, 1, 4, 3, 2, '5', '1', '3', '4', '2'], dtype=object)

In [9]:
data.groupby([TARGET_COL]).count()


,review_body
star_rating,
1,150441
2,97259
3,153660
4,259019
5,1040896
1,4566
2,3541
3,5999
4,11411


In [10]:
# Drop the outlier which is star_rating = "2012-12-21"
data = data[data.star_rating != "2012-12-21"]


In [11]:
# Remove nan valued rows
data = data[data[TARGET_COL].notnull()]


In [12]:
data.describe()


,star_rating,review_body
count,1766807,1766807
unique,10,1618522
top,5,Love it
freq,1040896,4288


In [13]:
# Convert all star rating to integer
data[TARGET_COL] = data.star_rating.astype(int)


In [14]:
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [15]:
data = data[data.review_body.notnull()]
data.head()


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [16]:
# There are no empty reviews
(data.review_body.str.len() <= 0).sum()


0

Now we can continue with the process.


## We select 20000 reviews randomly from each rating class.


In [17]:
N_SAMPLES = 20000

In [18]:
sampled_data = data.groupby(TARGET_COL, group_keys=False).apply(lambda x: x.sample(N_SAMPLES, random_state=RANDOM_SEED))
sampled_data.reset_index(inplace=True)
sampled_data.drop(columns=["index"], inplace=True)


In [19]:
sampled_data.head()

,star_rating,review_body
0,1,Too small even for the knuckles.
1,1,Did not fit right
2,1,This stupid kit has 16 gauge needles not 14gauge.
3,1,I would not suggest this item I bought the one...
4,1,I am sure that it will be lovely once I get it...


# Data Cleaning


In [20]:
avg_len_before_cleaning = sampled_data.review_body.str.len().mean()


In [21]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: pd.Series):
    return data.str.lower()


In [22]:
def remove_accented_characters(data: pd.Series):
    import unicodedata

    """Removes accented characters from the Series

    Args:
        data (pd.Series): Series of string

    Returns:
        _type_: pd.Series
    """
    import unicodedata

    return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))


In [23]:
def remove_html_encodings(data: pd.Series):
  return data.str.replace(r"&#\d+;", " ", regex=True)

In [24]:
def remove_html_tags(data: pd.Series):
  return data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

In [25]:
def remove_url(data: pd.Series):
  return data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

In [26]:
def remove_html_and_url(data: pd.Series):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (pd.Series): A Pandas series of type string

    Returns:
        _type_: pd.Series
    """
    # Remove HTML encodings
    data.str.replace(r"&#\d+;", " ", regex=True)

    # Remove HTML tags (both open and closed)
    data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

    # Remove URLs
    data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

    return data


In [27]:
! pip install emot

In [28]:
# Handle emoji
def convert_emoji_to_txt(data: pd.Series):
  from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

  EMO_TO_TXT_DICT = dict()
  for emot in UNICODE_EMOJI:
    EMO_TO_TXT_DICT[emot] = f" {re.sub(r',|:|_', '', UNICODE_EMOJI[emot])} "

  for emo in EMOTICONS_EMO:
    EMO_TO_TXT_DICT[emot] = f" {re.sub(r',| ', '', EMOTICONS_EMO[emo])} "

  def convert_emojis(text, emo_to_txt_dict):
    for emot in emo_to_txt_dict:
        text = text.replace(emot, emo_to_txt_dict[emot])
    return text

  return data.apply(lambda x: convert_emojis(x, EMO_TO_TXT_DICT))

In [29]:
def replace_digits_with_tag(data: pd.Series):  
  return data.str.replace(r"\d+", " NUM ", regex=True)

In [30]:
# Remove non-alphabetical characters
def remove_non_alpha_characters(data: pd.Series):
    return data.str.replace(r"_+|\\|[^a-zA-Z0-9\s]", " ", regex=True)


In [31]:
# Remove extra spaces
def remove_extra_spaces(data: pd.Series):
    return data.str.replace(r"^\s*|\s\s*", " ", regex=True)


In [32]:
# Install contractions package, if you don't have it
! pip install contractions

In [33]:
# Expanding contractions
def fix_contractions(data: pd.Series):
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    return data.apply(contraction_fixer)


In [34]:
# A dictionary containing the columns and a list of functions to perform on it in order
data_cleaning_pipeline = {
    DATA_COL: [
        convert_emoji_to_txt,
        to_lower,
        remove_accented_characters,
        remove_html_encodings,
        remove_html_tags,
        remove_url,
        fix_contractions,
        remove_non_alpha_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = sampled_data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = cleaned_data[col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[col] = temp_data.copy()


Starting: convert_emoji_to_txt
Ended: convert_emoji_to_txt
Starting: to_lower
Ended: to_lower
Starting: remove_accented_characters
Ended: remove_accented_characters
Starting: remove_html_encodings
Ended: remove_html_encodings
Starting: remove_html_tags
Ended: remove_html_tags
Starting: remove_url
Ended: remove_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpha_characters
Ended: remove_non_alpha_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [35]:
avg_len_after_cleaning = cleaned_data.review_body.str.len().mean()


In [36]:
f"{avg_len_before_cleaning},{avg_len_after_cleaning}"


'189.50703,185.32883'

# Pre-processing


## remove the stop words


In [39]:
avg_len_before_preprocessing = cleaned_data[DATA_COL].str.len().mean()


In [40]:
def tokenize(data: pd.Series):
    from nltk.tokenize import word_tokenize

    nltk.download("punkt")

    return data.apply(word_tokenize)


In [41]:
from typing import List, Set


def remove_stopwords(data: pd.Series):
    """Remove stop words using the NLTK stopwords dictionary

    Args:
        string (str): a document

    Returns:
        str: a document with stopwords removed
    """
    from nltk.corpus import stopwords

    nltk.download("stopwords")

    stopwords = set(stopwords.words())

    def remover(word_list: List[str], stopwords: Set[str]):
        return [word for word in word_list if not word in stopwords]

    return data.apply(lambda word_list: remover(word_list, stopwords))


## perform lemmatization


In [42]:
def lemmatize(data: pd.Series, consider_pos_tag: bool = True):
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer

    nltk.download("omw-1.4")

    # POS tagging
    def perform_nltk_pos_tag(data: pd.Series):
        from nltk import pos_tag

        nltk.download("averaged_perceptron_tagger")

        return data.apply(pos_tag)

    # Convert POS tag to wordnet pos tags
    def wordnet_pos_tagger(tag: str):
        if tag.startswith("J"):
            return wordnet.ADJ
        elif tag.startswith("V"):
            return wordnet.VERB
        elif tag.startswith("N"):
            return wordnet.NOUN
        elif tag.startswith("R"):
            return wordnet.ADV
        else:
            return None

    lemmatizer = WordNetLemmatizer()
    lemmatized = list()

    if consider_pos_tag:
        pos_tagged_data = data.copy()
        pos_tagged_data = perform_nltk_pos_tag(data)

        for row in pos_tagged_data:

            lemmatized_row = list()

            if consider_pos_tag:
                for word, tag in row:
                    wordnet_pos_tag = wordnet_pos_tagger(tag)

                    if wordnet_pos_tag is None:
                        lemmatized_row.append(word)
                    else:
                        result = lemmatizer.lemmatize(word, wordnet_pos_tag)
                        lemmatized_row.append(lemmatizer.lemmatize(word, wordnet_pos_tag))

            lemmatized.append(lemmatized_row)
    else:
        for row in data:
            lemmatized_row = list()

            for word in row:
                lemmatized_row.append(lemmatizer.lemmatize(word))

            lemmatized.append(lemmatized_row)

    return pd.Series(lemmatized)


In [43]:
# Concatenate lemmatized sentences back into one sentence
def concatenate(data: pd.Series):
    return data.apply(lambda words: " ".join(words))


In [44]:
preprocessing_pipeline = {DATA_COL: [tokenize, lemmatize, concatenate]}

# Run the pipeline
preprocessed_data = cleaned_data.copy()

# Process all the cleaning instructions
for col, pipeline in preprocessing_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = preprocessed_data[col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")

        if func.__name__ == "lemmatize":
            temp_data = func(temp_data, consider_pos_tag=True)
        else:
            temp_data = func(temp_data)

        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    preprocessed_data[col] = temp_data.copy()


Starting: tokenize


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ended: tokenize
Starting: lemmatize


[nltk_data] Downloading package omw-1.4 to /Users/aditya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aditya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Ended: lemmatize
Starting: concatenate
Ended: concatenate


In [45]:
preprocessed_data.head()

,star_rating,review_body
0,1,too small even for the knuckle
1,1,do not fit right
2,1,this stupid kit have 16 gauge needle not 14gauge
3,1,i would not suggest this item i buy the one wi...
4,1,i be sure that it will be lovely once i get it...


In [64]:
avg_len_after_preprocessing = preprocessed_data[DATA_COL].str.len().mean()


: 

In [48]:
f"{avg_len_before_preprocessing},{avg_len_after_preprocessing}"


'185.32883,176.85228'

# TF-IDF Feature Extraction


In [51]:
# Remove empty reviews
preprocessed_data = preprocessed_data[preprocessed_data[DATA_COL].str.len() != 0]

In [56]:
final_data = preprocessed_data.copy()

In [57]:
# Split the data 80-20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_data, test_size=0.2, stratify=final_data[TARGET_COL], random_state=RANDOM_SEED)


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

nltk.download("punkt")

vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
vectorizer.fit(final_data[DATA_COL])

X_tfidf_train = vectorizer.transform(train[DATA_COL])
X_tfidf_test = vectorizer.transform(test[DATA_COL])
y_train = train[TARGET_COL]
y_test = test[TARGET_COL]


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544-assignment-01/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Helper Functions


In [59]:
def calc_metrics(y_true, y_pred):
    from sklearn.metrics import precision_score, recall_score, f1_score

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    for rating_precision, rating_recall, rating_f1 in zip(precision, recall, f1):
        print(f"{rating_precision},{rating_recall},{rating_f1}")

    print(f"{np.mean(precision)},{np.mean(recall)},{np.mean(f1)}")


# Perceptron


In [60]:
from sklearn.linear_model import Perceptron

# clf = Perceptron(max_iter=1000, alpha=0.5, random_state=RANDOM_SEED) # 0.43115027900835684
# clf = Perceptron(max_iter=1000, alpha=0.5, random_state=RANDOM_SEED, tol=1e-5) # 0.4336870195755223
# clf = Perceptron(max_iter=4000, alpha=0.5, random_state=RANDOM_SEED, tol=1e-5, early_stopping=True) # 0.4343847796358854
# clf = Perceptron(max_iter=8000, alpha=0.01, random_state=RANDOM_SEED, tol=1e-5, early_stopping=True, class_weight="balanced") # 0.4343847796358854

# class_weight = {1: 0.29525, 2: 16.95725, 3: 8.9525, 4: 1, 5: 0.7385} # 0.45103975891511683
# class_weight = {1: 0.29525, 2: 7.8048, 3: 8.4525, 4: 1, 5: 0.9385}
class_weight = {1: 0.9525, 2: 1.99825, 3: 1.9225, 4: 0.625, 5: 0.8585}
clf = Perceptron(max_iter=8000, alpha=0.012, random_state=RANDOM_SEED, tol=1e-4, early_stopping=True, class_weight="balanced") # 0.45103975891511683

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

# print("Train ...")
# calc_metrics(y_train, clf.predict(X_tfidf_train))
# print("Test...")
calc_metrics(y_test, y_pred)

0.5179242918024568,0.5166291572893223,0.5172759138708063
0.3148377508050533,0.31775,0.31628717183028493
0.3168876755070203,0.40625,0.3560473269062227
0.37042925278219396,0.3495,0.359660406483149
0.6373411534701857,0.48924462231115556,0.5535587943964907
0.43148402487338205,0.4158747559200956,0.4205659226973907


# SVM


In [61]:
from sklearn.svm import LinearSVC

# class_weight = {1: 0.95, 2: 1.7, 3: 1.65, 4: 1, 5: 0.6525} # 0.5250080879095086
# class_weight = {1: 0.95, 2: 1.65, 3: 1.85, 4: 1, 5: 0.6525} # 0.5265625011400694
# class_weight = {1: 0.95, 2: 1.725, 3: 2.85, 4: 1, 5: 0.6525} # 0.5283863569082248
# class_weight = {1: 0.95, 2: 1.725, 3: 2.85, 4: 1, 5: 0.6525} # 0.5298186572543087
# class_weight = {1: 0.95, 2: 1.725, 3: 2.925, 4: 1, 5: 0.8525} # 0.5298707666039373
# class_weight = {1: 0.95, 2: 1.725, 3: 2.925, 4: 1, 5: 0.9585} # 0.5305190760270104
# class_weight = {1: 0.95, 2: 1.8525, 3: 2.925, 4: 1, 5: 0.9585} # 0.5318580010845554
# class_weight = {1: 0.95, 2: 1.8525, 3: 3.3525, 4: 1, 5: 0.9585} # 0.5328127772891916
# class_weight = {1: 0.95, 2: 1.9725, 3: 3.3525, 4: 1, 5: 0.9585} # 0.534678946516937
# class_weight = {1: 0.95, 2: 2.3725, 3: 3.3525, 4: 1, 5: 0.9585} # 0.5378650442928313
# class_weight = {1: 0.95, 2: 2.85725, 3: 3.5525, 4: 1, 5: 0.9585} # 0.5403434839237587
# class_weight = {1: 0.95, 2: 4.85725, 3: 3.8525, 4: 1, 5: 0.9585} # 0.5474152267990974
# class_weight = {1: 0.7625, 2: 4.85725, 3: 3.8525, 4: 1, 5: 0.9585} # 0.551693399374575
# class_weight = {1: 0.7625, 2: 5.25725, 3: 3.9525, 4: 1, 5: 0.9585} # 0.5519983833002682
# class_weight = {1: 0.7625, 2: 5.95725, 3: 3.9525, 4: 1, 5: 0.9585} # 0.5530668437190186
# class_weight = {1: 0.5625, 2: 5.95725, 3: 4.2525, 4: 1, 5: 0.9585} # 0.5634729752537446
# class_weight = {1: 0.5625, 2: 6.95725, 3: 4.6525, 4: 1, 5: 0.8585} # 0.5656444257835423
# class_weight = {1: 0.2625, 2: 6.95725, 3: 4.6525, 4: 1, 5: 0.8585} # 0.5732672392507041
class_weight = {1: 0.9525, 2: 1.99825, 3: 1.9225, 4: 0.625, 5: 0.8585} # 0.5746383318772281

clf = LinearSVC(dual=False, C=0.1, max_iter=1000, class_weight=class_weight, random_state=RANDOM_SEED)

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calc_metrics(y_test, y_pred)

0.6652977412731006,0.48612153038259565,0.5617685305591678
0.38524890190336747,0.52625,0.444843617920541
0.3793507362784471,0.56675,0.4544907778668805
0.6078431372549019,0.14725,0.2370698329643791
0.6618088298328333,0.7723861930965483,0.7128347183748847
0.53990986930853,0.4997515446958287,0.4822014955371706


# Logistic Regression


In [62]:
from sklearn.linear_model import LogisticRegression

# class_weight = {1: 1, 2: 1, 3: 1, 4: 1, 5: 1.1024} # 0.5172256481829293
class_weight = {1: 0.9525, 2: 1.99825, 3: 1.9225, 4: 0.625, 5: 0.8585} 

clf = LogisticRegression(penalty='l2', solver="saga", max_iter=200, multi_class="multinomial", C=0.1024, random_state=RANDOM_SEED, class_weight=class_weight)

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

# print("Train ...")
# calc_metrics(y_train, clf.predict(X_tfidf_train))
# print("Test...")
calc_metrics(y_test, y_pred)

0.7653631284916201,0.3083270817704426,0.439572192513369
0.3659177122847972,0.627,0.4621337755666114
0.3540203525870718,0.6175,0.4500318848501412
0.6307339449541285,0.06875,0.12398557258791706
0.6792910900704054,0.6998499249624812,0.6894172723912775
0.5590652456776046,0.4642854013465848,0.43302813958186326


# Naive Bayes


In [63]:
from sklearn.naive_bayes import MultinomialNB

class_prior = [0.2, 0.2048, 0.2, 0.2, 0.22476]

# clf = MultinomialNB(alpha=1.064) # 0.4952600291865206
# clf = MultinomialNB(alpha=2.5) # 0.5024158722296946
# clf = MultinomialNB(alpha=3.9086) # 0.5065314917640114
# clf = MultinomialNB(alpha=32.7886) # 0.5124296411150511
clf = MultinomialNB(alpha=32.8824, class_prior=class_prior) # 0.5126996219647234

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

# print("Train ...")
# calc_metrics(y_train, clf.predict(X_tfidf_train))
# print("Test...")
calc_metrics(y_test, y_pred)

0.6752488843117062,0.4918729682420605,0.5691550925925926
0.38682055100521223,0.5195,0.44344857020913364
0.40814814814814815,0.41325,0.41068322981366456
0.4389505549949546,0.435,0.4369663485685585
0.6687398593834505,0.6185592796398199,0.6426715176715176
0.5155815995686943,0.49563644957637615,0.5005849517710934
